In [1]:
import igl
import meshplot as mp
import numpy as np
# mp.offline() # one can manually switch to offline mode with this command https://skoch9.github.io/meshplot/tutorial/

In [2]:
# loading and plotting mesh
vert, tri = igl.read_triangle_mesh("./igea.ply")
plot_mesh = mp.plot(vert, tri)
#print('vertices = ',vert)
##print('vertices_shape =', vert.shape)
#print('triangles = ', tri)
#print('triangles_shape =',tri.shape)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.021590…

In [3]:
# implement away
# do not use other functions than provided by basic python or numpy 
# (especially not the gaussian curvature function provided by igl)
## 
# algorithm:
# iterating through each triangle in triangles array, calculating area and theta values corresponding to three vertices 
# in each triangle and adding the corresponding values to corresponding elements in array created and calculating
# gaussian curvature values from this array. 
##   
def gaussian(vertices, triangles):
    
    # create an array to store area and theta values for each vertex
    arr = np.zeros((len(vertices),2))
    
    # calculate theta and area for each vertex in each triangle  
    for tri in triangles:
        
        # lengths of sides in each triangle using norm method
        a = np.linalg.norm(vertices[tri[0]] - vertices[tri[1]])
        b = np.linalg.norm(vertices[tri[1]] - vertices[tri[2]])
        c = np.linalg.norm(vertices[tri[2]] - vertices[tri[0]])

        # theta values in each triangle for each vertex using cosine triangle rule
        theta_a = np.arccos((-a*a + b*b + c*c)/ (2*c*b))
        theta_b = np.arccos((a*a - b*b + c*c)/ (2*a*c))
        theta_c = np.arccos((a*a + b*b - c*c)/ (2*a*b))

        # area of triangle using cross product 
        area = (1/3)*(0.5*a*b*(np.sin(theta_c)))

        # add to corresponding values of theta in array arr
        arr[tri[0]][0] += theta_b
        arr[tri[1]][0] += theta_c
        arr[tri[2]][0] += theta_a

        # add to corresponding values of area in array arr
        arr[tri[0]][1] += area
        arr[tri[1]][1] += area
        arr[tri[2]][1] += area
        
    # using broadcasting to calculate gaussian curvature  
    g = (2*np.pi - arr[:, 0])/arr[:, 1]
    return g
        

In [4]:
g = gaussian(vert, tri)

In [5]:
# plot curvature function on the surface (cutting off extreme values)
colors = mp.utils.get_colors(g, colormap='viridis', normalize=False, vmin=-200, vmax=200)
mp.plot(vert, tri, colors)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.021590…